# Project 2 - Team 6: 6in6_Transform_NumbeoAPI
Author: Jose Tomines<br>
Date: 2019-01-16<br>
Comments:<br>
1) Determine cities & their ids from Numbeo<br>
2) Determine cities, their provinces, and population data from Stats Canada csv file<br>
3) Clean data to ensure clean merge<br>
4) Get Cost of Living information for each city and append columns to city dataframe<br>
5) Aggregate city data into provinces

In [11]:
# Dependencies
import pandas as pd
import requests
from pprint import pprint
import json
import time
import csv

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

## Extract Cities from Numbeo API

In [2]:
# Build query URL for each page
query_url = "https://www.numbeo.com/api/cities?api_key=my6bstu4nupdl5"

# get data from API
city_response = requests.get(query_url).json()

In [3]:
# Data to store from the json
city_id=[]
city_name=[]
city_lat=[]
city_lng=[]

rows = int(len(city_response['cities']))

# create arrays for cities in Canada
for x in range(len(city_response['cities'])):
    rowText = city_response['cities'][x]

    if rowText['country'] == 'Canada':

        cityId = rowText['city_id']
        cityName = rowText['city']
        
        if 'longitude' in rowText:
            cityLng = rowText['longitude']
        else:
            cityLng = None
            
        if 'latitude' in rowText:
            cityLat = rowText['latitude']
        else:
            cityLat = None            
        
        city_id.append(cityId)
        city_name.append(cityName)
        city_lng.append(cityLng)
        city_lat.append(cityLat)
        
# create city dataframe
city_data = {'city_id': city_id,
             'city_name': city_name,
             'longitude': city_lng,
             'latitude': city_lat
            }

city_orig_df = pd.DataFrame(city_data)

# Preview Data
city_orig_df.head()

,city_id,city_name,longitude,latitude
0,33365,North Vancouver,-123.072414,49.319982
1,33399,Cochrane,-114.467860,51.190988
2,33600,Slave Lake,-114.771118,55.286683
3,33745,"Tottenham, ON",-79.805595,44.022484
4,33754,"Boisbriand, QC, Canada",-73.838373,45.612634


In [4]:
# Some city names had the province and even the country as part of the city name
tempcity = city_orig_df['city_name'].str.split(", ", n=2, expand = True)
tempcity = tempcity.rename(columns = {0: 'city', 1:'province'})
tempcity.province.unique()

array([None, 'ON', 'QC', 'Ont', 'AB', 'Ontario', 'NT', 'NS', 'BC', 'SK',
       'MB', 'NL', 'NU', 'PE', 'NB', 'Manitoba'], dtype=object)

In [5]:
# Ensure all provinces are based on approved 2 character code
tempcity.loc[tempcity['province'] == 'Ont', 'province'] = 'ON'
tempcity.loc[tempcity['province'] == 'Ontario', 'province'] = 'ON'
tempcity.loc[tempcity['province'] == 'Manitoba', 'province'] = 'MB'
tempcity.province.unique()

array([None, 'ON', 'QC', 'AB', 'NT', 'NS', 'BC', 'SK', 'MB', 'NL', 'NU',
       'PE', 'NB'], dtype=object)

In [6]:
# Preview of tempcity dataframe
tempcity.head()

,city,province,2
0,North Vancouver,None,None
1,Cochrane,None,None
2,Slave Lake,None,None
3,Tottenham,ON,None
4,Boisbriand,QC,Canada
5,Leamington,None,None
6,Drummondville,None,None
7,Summerland,None,None
8,Armstrong,None,None
9,Marathon,None,None


In [50]:
# Since same # of rows, and visual inspection shows similar columns, join the two tables
city_df = pd.merge(city_orig_df, tempcity, left_index = True, right_index = True)
city_df = city_df.reindex_axis(['city_id', 'city', 'longitude', 'latitude', 'city_name', 'province'], axis=1)
city_df = city_df.rename(columns = {"city_name" : "orig_city_name", "city" : "city_name", "province": "prov_from_name"})

# Some city names do not match precisely with the city names from the Stats Can data used below.  Changing the names to match
city_df.loc[city_df.city_id == 34395, 'city_name'] = 'Barrhead'
city_df.loc[city_df.city_id == 34640, 'city_name'] = 'Sainte-Agathe-des-Monts'
city_df.loc[city_df.city_id == 2364, 'city_name'] = "St. John's"
city_df.loc[city_df.city_id == 35551, 'city_name'] = 'Thorold'
city_df.loc[city_df.city_id == 7611, 'city_name'] = 'Saint-Sauveur'
city_df.loc[city_df.city_id == 8034, 'city_name'] = 'St. Catherines'
city_df.loc[city_df.city_id == 8547, 'city_name'] = 'Nelson'
city_df.loc[city_df.city_id == 9903, 'city_name'] = 'Lake Louise'
city_df.loc[city_df.city_id == 9961, 'city_name'] = 'New Glasgow'
city_df.loc[city_df.city_id == 10069, 'city_name'] = 'Dawson'
city_df.loc[city_df.city_id == 10568, 'city_name'] = 'Duncan'
city_df.loc[city_df.city_id == 10979, 'city_name'] = 'Rimouski'
city_df.loc[city_df.city_id == 13301, 'city_name'] = 'Deschambault-Grondines'
city_df.loc[city_df.city_id == 13303, 'city_name'] = 'Ferme-Neuve'
city_df.loc[city_df.city_id == 13310, 'city_name'] = 'La Dore'
city_df.loc[city_df.city_id == 13312, 'city_name'] = 'La Peche'

city_df.head()

,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,None
1,33399,Cochrane,-114.467860,51.190988,Cochrane,None
2,33600,Slave Lake,-114.771118,55.286683,Slave Lake,None
3,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON
4,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC


## Load Stats Canada csv for population data per locaton

In [46]:
# File to Load
citiesFile = "../1_Input/T301EN.CSV"

# Load to dataframe
statscan_cities_data = pd.read_csv(citiesFile, encoding="latin-1")

# Rename columns for columns to be saved
ca_cities = statscan_cities_data.rename(columns = {"Geographic name, english" : "name",
                            "CSD type, english" : "type",
                            "Province / territory, english" : "province_name",
                            "Total private dwellings, 2016" : "private_dwellings",
                            "Land area in square kilometres, 2016" : "land_area_sqkm",
                            "Population density per square kilometre, 2016" : "pop_density_per_sqkm",
                            "Population, 2016" : "population"
                           })

ca_cities = ca_cities.drop(columns = ["Geographic code",
                                      "Geographic name, french",
                                      "CSD type, french",
                                      "Province / territory, french",
                                      "Geographic code, Province / territory",
                                      "Geographic code, Census division",
                                      "Geographic code, Census metropolitan area / census agglomeration",
                                      "2011 adjusted total private dwellings flag",
                                      "Total private dwellings, % change",
                                      "Private dwellings occupied by usual residents, 2011",
                                      "2011 adjusted private dwellings occupied by usual residents flag",
                                      "Private dwellings occupied by usual residents, % change",
                                      "National population rank, 2016",
                                      "Provincial/territorial population rank, 2016",
                                      "Incompletely enumerated Indian reserves and Indian settlements, 2016",
                                      "Population, 2011",
                                      "2011 adjusted population flag",
                                      "Incompletely enumerated Indian reserves and Indian settlements, 2011",
                                      "2011 population review or received update flag",
                                      "Private dwellings occupied by usual residents, 2016",
                                      "Total private dwellings, 2011",
                                      "Population, % change"
                                     ])

# Rename because of the accents
ca_cities.iloc[861, ca_cities.columns.get_loc('name')] = 'Gaspe'
ca_cities.iloc[1127, ca_cities.columns.get_loc('name')] = 'Quebec'
ca_cities.iloc[1286, ca_cities.columns.get_loc('name')] = 'Trois-Rivieres'
ca_cities.iloc[1639, ca_cities.columns.get_loc('name')] = 'Montreal'
ca_cities.iloc[1753, ca_cities.columns.get_loc('name')] = 'Sainte-Adele'
ca_cities.iloc[1841, ca_cities.columns.get_loc('name')] = 'La Peche'
ca_cities.iloc[1980, ca_cities.columns.get_loc('name')] = 'La Dore'
ca_cities.iloc[2052, ca_cities.columns.get_loc('name')] = 'Sept-Iles'
ca_cities.iloc[2413, ca_cities.columns.get_loc('name')] = 'Minden'
ca_cities.iloc[2904, ca_cities.columns.get_loc('name')] = 'Flin Flon'
ca_cities.iloc[4081, ca_cities.columns.get_loc('name')] = 'Lloydminster'

ca_cities.head()

,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm
0,"Division No. 1, Subd. V",Subdivision of unorganized,Newfoundland and Labrador,36.0,140.0,894.15,0.0
1,Portugal Cove South,Town,Newfoundland and Labrador,150.0,91.0,1.14,131.5
2,Trepassey,Town,Newfoundland and Labrador,481.0,356.0,55.88,8.6
3,St. Shott's,Town,Newfoundland and Labrador,66.0,52.0,1.13,58.4
4,"Division No. 1, Subd. U",Subdivision of unorganized,Newfoundland and Labrador,1625.0,1630.0,754.35,2.2


In [47]:
ca_cities.province_name.unique()

array(['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan',
       'Alberta', 'British Columbia', 'Yukon', 'Northwest Territories',
       'Nunavut', nan], dtype=object)

In [48]:
# Assign standard province abbreviation to province column
ca_cities.loc[ca_cities['province_name'] == 'Manitoba', 'province'] = 'MB'
ca_cities.loc[ca_cities['province_name'] == 'Newfoundland and Labrador', 'province'] = 'NL'
ca_cities.loc[ca_cities['province_name'] == 'Yukon', 'province'] = 'YT'
ca_cities.loc[ca_cities['province_name'] == 'Québec', 'province'] = 'QC'
ca_cities.loc[ca_cities['province_name'] == 'Quebec', 'province'] = 'QC'
ca_cities.loc[ca_cities['province_name'] == 'Saskatchewan', 'province'] = 'SK'
ca_cities.loc[ca_cities['province_name'] == 'Nova Scotia', 'province'] = 'NS'
ca_cities.loc[ca_cities['province_name'] == 'Alberta', 'province'] = 'AB'
ca_cities.loc[ca_cities['province_name'] == 'Ontario', 'province'] = 'ON'
ca_cities.loc[ca_cities['province_name'] == 'British Columbia', 'province'] = 'BC'
ca_cities.loc[ca_cities['province_name'] == 'Northwest Territories', 'province'] = 'NT'
ca_cities.loc[ca_cities['province_name'] == 'Nunavut', 'province'] = 'NU'
ca_cities.loc[ca_cities['province_name'] == 'New Brunswick', 'province'] = 'NB'
ca_cities.loc[ca_cities['province_name'] == 'Prince Edward Island', 'province'] = 'PE'

ca_cities.head()

,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province
0,"Division No. 1, Subd. V",Subdivision of unorganized,Newfoundland and Labrador,36.0,140.0,894.15,0.0,NL
1,Portugal Cove South,Town,Newfoundland and Labrador,150.0,91.0,1.14,131.5,NL
2,Trepassey,Town,Newfoundland and Labrador,481.0,356.0,55.88,8.6,NL
3,St. Shott's,Town,Newfoundland and Labrador,66.0,52.0,1.13,58.4,NL
4,"Division No. 1, Subd. U",Subdivision of unorganized,Newfoundland and Labrador,1625.0,1630.0,754.35,2.2,NL


## Merge DataFrames to have locations with population data and province codes

In [53]:
# Assign provinces to each of the city in the city_df
combined_cities_df = pd.merge(city_df, ca_cities, how='left', left_on=['city_name'], right_on=['name'])

# Remove duplicate rows

combined_cities_df

,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,None,North Vancouver,District municipality,British Columbia,85935.0,32624.0,160.76,534.6,BC
1,33365,North Vancouver,-123.072414,49.319982,North Vancouver,None,North Vancouver,City,British Columbia,52898.0,26426.0,11.85,4465.1,BC
2,33399,Cochrane,-114.467860,51.190988,Cochrane,None,Cochrane,Town,Ontario,5321.0,2510.0,539.12,9.9,ON
3,33399,Cochrane,-114.467860,51.190988,Cochrane,None,Cochrane,Town,Alberta,25853.0,10225.0,29.83,866.7,AB
4,33600,Slave Lake,-114.771118,55.286683,Slave Lake,None,Slave Lake,Town,Alberta,6651.0,2762.0,14.44,460.5,AB
5,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC,Boisbriand,Ville,Quebec,26884.0,10537.0,27.82,966.5,QC
7,33771,Leamington,-82.599887,42.053163,Leamington,None,Leamington,Municipality,Ontario,27595.0,10726.0,262.01,105.3,ON
8,33831,Drummondville,-72.484282,45.880291,Drummondville,None,Drummondville,Ville,Quebec,75423.0,35402.0,247.15,305.2,QC
9,33836,Summerland,-119.677778,49.600556,Summerland,None,Summerland,District municipality,British Columbia,11615.0,5240.0,74.08,156.8,BC


In [58]:
# NOTE this section can be removed if the duplicate rows are automated in the previous cell
# Saved to file so that I can manually remove duplicates
combined_cities_df.to_csv('../1_Extracted_Inputs/combined_city.csv', index=False, encoding='utf-8')

# Loaded manually cleaned up csv to go on...
# File to Load
citiesFile = "../1_Extracted_Inputs/combined__city.csv"

# Load to dataframe
combined_cities_df = pd.read_csv(citiesFile)

combined_cities_df.head()

,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,NaN,North Vancouver,City,British Columbia,52898.0,26426.0,11.85,4465.1,BC
1,33399,Cochrane,-114.467860,51.190988,Cochrane,NaN,Cochrane,Town,Alberta,25853.0,10225.0,29.83,866.7,AB
2,33600,Slave Lake,-114.771118,55.286683,Slave Lake,NaN,Slave Lake,Town,Alberta,6651.0,2762.0,14.44,460.5,AB
3,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC,Boisbriand,Ville,Quebec,26884.0,10537.0,27.82,966.5,QC


In [59]:
# Some lines did not match up with stats can data, will add the province
combined_cities_df.loc[combined_cities_df.city_id == 7521, 'province'] = 'AB'
combined_cities_df.loc[combined_cities_df.city_id == 9903, 'province'] = 'AB'
combined_cities_df.loc[combined_cities_df.city_id == 11094, 'province'] = 'AB'
combined_cities_df.loc[combined_cities_df.city_id == 12810, 'province'] = 'AB'
combined_cities_df.loc[combined_cities_df.city_id == 7926, 'province'] = 'BC'
combined_cities_df.loc[combined_cities_df.city_id == 8803, 'province'] = 'BC'
combined_cities_df.loc[combined_cities_df.city_id == 11339, 'province'] = 'MB'
combined_cities_df.loc[combined_cities_df.city_id == 9939, 'province'] = 'NS'
combined_cities_df.loc[combined_cities_df.city_id == 10230, 'province'] = 'NS'
combined_cities_df.loc[combined_cities_df.city_id == 33745, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 33871, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 34041, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 34380, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 34478, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 35080, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 2348, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 2367, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 35360, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 35394, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 8287, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 8817, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 9667, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 10110, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 10464, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 11868, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 12436, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 13302, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 13306, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 13317, 'province'] = 'ON'
combined_cities_df.loc[combined_cities_df.city_id == 9204, 'province'] = 'QC'

combined_cities_df.head()


,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,NaN,North Vancouver,City,British Columbia,52898.0,26426.0,11.85,4465.1,BC
1,33399,Cochrane,-114.467860,51.190988,Cochrane,NaN,Cochrane,Town,Alberta,25853.0,10225.0,29.83,866.7,AB
2,33600,Slave Lake,-114.771118,55.286683,Slave Lake,NaN,Slave Lake,Town,Alberta,6651.0,2762.0,14.44,460.5,AB
3,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON
4,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC,Boisbriand,Ville,Quebec,26884.0,10537.0,27.82,966.5,QC


# Get price data and index for each location

In [61]:
# Create columns to hold price and index data
complete_city_df = combined_cities_df
complete_city_df['apt_mthly_rent'] = float('nan')
complete_city_df['property_price_sqft'] = float('nan')
complete_city_df['ave_mthly_net_salary'] = float('nan')
complete_city_df['quality_of_life_idx'] = float('nan')
complete_city_df['cost_of_living_idx'] = float('nan')

complete_city_df.head()

,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province,apt_mthly_rent,property_price_sqft,ave_mthly_net_salary,quality_of_life_idx,cost_of_living_idx
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,NaN,North Vancouver,City,British Columbia,52898.0,26426.0,11.85,4465.1,BC,NaN,NaN,NaN,NaN,NaN
1,33399,Cochrane,-114.467860,51.190988,Cochrane,NaN,Cochrane,Town,Alberta,25853.0,10225.0,29.83,866.7,AB,NaN,NaN,NaN,NaN,NaN
2,33600,Slave Lake,-114.771118,55.286683,Slave Lake,NaN,Slave Lake,Town,Alberta,6651.0,2762.0,14.44,460.5,AB,NaN,NaN,NaN,NaN,NaN
3,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON,NaN,NaN,NaN,NaN,NaN
4,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC,Boisbriand,Ville,Quebec,26884.0,10537.0,27.82,966.5,QC,NaN,NaN,NaN,NaN,NaN


In [62]:
for row in range(len(complete_city_df)):
    cityID = complete_city_df.iloc[row, 0]
    query_url = "https://www.numbeo.com/api/city_prices?api_key=my6bstu4nupdl5&city_id=" + str(cityID)
    resp = requests.get(query_url).json()
    
    apt_rent = float('nan')
    price_sqft = float('nan')
    ave_mthly_net_salary = float('nan')
    
    for x in range(len(resp['prices'])):
        if resp['prices'][x]['item_id'] == 28:
            apt_rent = resp['prices'][x]['average_price']
        if resp['prices'][x]['item_id'] == 100:
            price_sqft = resp['prices'][x]['average_price']
        if resp['prices'][x]['item_id'] == 105:
            ave_mthly_net_salary = resp['prices'][x]['average_price']

    complete_city_df.iloc[row, complete_city_df.columns.get_loc('apt_mthly_rent')] = apt_rent
    complete_city_df.iloc[row, complete_city_df.columns.get_loc('property_price_sqft')] =  price_sqft
    complete_city_df.iloc[row, complete_city_df.columns.get_loc('ave_mthly_net_salary')] = ave_mthly_net_salary
    
complete_city_df.head()

,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province,apt_mthly_rent,property_price_sqft,ave_mthly_net_salary,quality_of_life_idx,cost_of_living_idx
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,NaN,North Vancouver,City,British Columbia,52898.0,26426.0,11.85,4465.1,BC,3805.0,9592.89513,4750.000000,NaN,NaN
1,33399,Cochrane,-114.467860,51.190988,Cochrane,NaN,Cochrane,Town,Alberta,25853.0,10225.0,29.83,866.7,AB,2400.0,NaN,2250.000000,NaN,NaN
2,33600,Slave Lake,-114.771118,55.286683,Slave Lake,NaN,Slave Lake,Town,Alberta,6651.0,2762.0,14.44,460.5,AB,2200.0,NaN,4958.333333,NaN,NaN
3,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON,NaN,NaN,NaN,NaN,NaN
4,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC,Boisbriand,Ville,Quebec,26884.0,10537.0,27.82,966.5,QC,NaN,NaN,3916.500000,NaN,NaN


In [76]:
# Get city indices data

for row in range(len(complete_city_df)):
    cityID = complete_city_df.iloc[row, 0]
    query_url = "https://www.numbeo.com/api/indices?api_key=my6bstu4nupdl5&city_id=" + str(cityID)
    resp = requests.get(query_url).json()
    
    quality_of_life_idx = float('nan')
    cost_of_living_idx = float('nan')
    
    if 'cpi_index' in resp:
        cost_of_living_idx = resp['cpi_index']
    else:
        cost_of_living_idx = float('nan')

    if 'quality_of_life_index' in resp:
        quality_of_life_idx = resp['quality_of_life_index']
    else:
        quality_of_life_idx = float('nan')            

    complete_city_df.iloc[row, complete_city_df.columns.get_loc('quality_of_life_idx')] = quality_of_life_idx
    complete_city_df.iloc[row, complete_city_df.columns.get_loc('cost_of_living_idx')] = cost_of_living_idx

complete_city_df.head()

,city_id,city_name,longitude,latitude,orig_city_name,prov_from_name,name,type,province_name,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,province,apt_mthly_rent,property_price_sqft,ave_mthly_net_salary,quality_of_life_idx,cost_of_living_idx
0,33365,North Vancouver,-123.072414,49.319982,North Vancouver,NaN,North Vancouver,City,British Columbia,52898.0,26426.0,11.85,4465.1,BC,3805.0,9592.89513,4750.000000,NaN,NaN
1,33399,Cochrane,-114.467860,51.190988,Cochrane,NaN,Cochrane,Town,Alberta,25853.0,10225.0,29.83,866.7,AB,2400.0,NaN,2250.000000,NaN,NaN
2,33600,Slave Lake,-114.771118,55.286683,Slave Lake,NaN,Slave Lake,Town,Alberta,6651.0,2762.0,14.44,460.5,AB,2200.0,NaN,4958.333333,NaN,NaN
3,33745,Tottenham,-79.805595,44.022484,"Tottenham, ON",ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON,NaN,NaN,NaN,NaN,NaN
4,33754,Boisbriand,-73.838373,45.612634,"Boisbriand, QC, Canada",QC,Boisbriand,Ville,Quebec,26884.0,10537.0,27.82,966.5,QC,NaN,NaN,3916.500000,NaN,NaN


In [78]:
# Saving table so that it can be used for data visualization
complete_city_df.to_csv('../1_Extracted_Inputs/complete_city_df.csv', index=False, encoding='utf-8')

## Aggregate data to build Province table

In [71]:
# Group Stat Can data into provinces
ca_province = ca_cities.groupby(["province"]).agg({"population" : "sum",
                                                   "private_dwellings" : "sum",
                                                   "land_area_sqkm" : "sum"
                                                  })

ca_province["pop_density_per_sqkm"] = ca_province["population"] / ca_province["land_area_sqkm"]

ca_province

,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm
province,,,,
AB,4067175.0,1654129.0,640330.37,6.351682
BC,4648055.0,2063417.0,922503.15,5.038525
MB,1278365.0,539748.0,552371.04,2.314323
NB,747101.0,359721.0,71388.79,10.465243
NL,519716.0,265739.0,370514.18,1.402689
NS,923598.0,458568.0,52942.33,17.445360
NT,41786.0,17666.0,1143793.84,0.036533
NU,35944.0,11433.0,1877778.53,0.019142
ON,13448494.0,5598391.0,908699.40,14.799717


In [79]:
# group city price index data into provinces
grp_city = complete_city_df.groupby(["province"]).agg({"apt_mthly_rent" : "mean",
                                                       "property_price_sqft" : "mean",
                                                       "ave_mthly_net_salary" : "mean",
                                                       "quality_of_life_idx" : "mean",
                                                       "cost_of_living_idx" : "mean"
                                                      })

grp_city

,apt_mthly_rent,property_price_sqft,ave_mthly_net_salary,quality_of_life_idx,cost_of_living_idx
province,,,,,
AB,1730.873599,3444.570786,4092.725040,170.538382,69.251325
BC,1956.254276,4872.388237,3242.889212,180.940242,66.211790
MB,1366.346154,2865.333076,2326.941667,147.540275,60.870704
NB,1092.149170,1338.303892,2751.376036,173.113767,67.561323
NL,1560.555556,3426.391042,3239.559524,160.851640,74.679354
NS,1295.973810,2119.426232,3228.612500,176.229324,76.419870
NT,1911.111111,3733.684323,4262.520833,NaN,82.890726
NU,3600.000000,10763.910417,2700.000000,NaN,NaN
ON,1745.347523,4203.015516,3403.371819,175.669800,66.349085


In [80]:
# Merge dataframes together
province_df = ca_province.join(grp_city, how='outer')
province_df

,population,private_dwellings,land_area_sqkm,pop_density_per_sqkm,apt_mthly_rent,property_price_sqft,ave_mthly_net_salary,quality_of_life_idx,cost_of_living_idx
province,,,,,,,,,
AB,4067175.0,1654129.0,640330.37,6.351682,1730.873599,3444.570786,4092.725040,170.538382,69.251325
BC,4648055.0,2063417.0,922503.15,5.038525,1956.254276,4872.388237,3242.889212,180.940242,66.211790
MB,1278365.0,539748.0,552371.04,2.314323,1366.346154,2865.333076,2326.941667,147.540275,60.870704
NB,747101.0,359721.0,71388.79,10.465243,1092.149170,1338.303892,2751.376036,173.113767,67.561323
NL,519716.0,265739.0,370514.18,1.402689,1560.555556,3426.391042,3239.559524,160.851640,74.679354
NS,923598.0,458568.0,52942.33,17.445360,1295.973810,2119.426232,3228.612500,176.229324,76.419870
NT,41786.0,17666.0,1143793.84,0.036533,1911.111111,3733.684323,4262.520833,NaN,82.890726
NU,35944.0,11433.0,1877778.53,0.019142,3600.000000,10763.910417,2700.000000,NaN,NaN
ON,13448494.0,5598391.0,908699.40,14.799717,1745.347523,4203.015516,3403.371819,175.669800,66.349085


In [1]:
# Saving table so that it can be used for data visualization
province_df.to_csv('../1_Extracted_Inputs/province_df.csv', index=True, encoding='utf-8')

NameError: name 'province_df' is not defined